In [ ]:
import os
os.chdir('/Users/tomharrison/Documents/Projects/TourDeFrance2020')

In [ ]:
import pandas as pd
#import json
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def readstagedata(rider, stage):
    df = pd.read_json("./data/{0}_stage_{1}.json".format(rider, stage))
    
    #creating new columns with more intuitive units
    df['Speed'] = df['velocity_smooth'] * 3.6  # m/s to kph
    df['Distance'] = df['distance'] / 1000      # m to km
    
    #remove old columns with less intuitive units
    df = df.drop(['distance', 'velocity_smooth'], axis = 1)
    
    #rename columns
    df = df.rename(columns = {"temp":"Temperature", "grade_smooth":"Gradient", "cadence":"Cadence", "altitude":"Altitude", "watts_calc":"Estimated watts", "time":"Time"})
    
    #reorder columns for easier visualisation
    columnstitles = ['Time', 'Distance', 'Speed', 'Altitude', 'Gradient', 'Estimated watts', 'Temperature', 'Cadence', 'latlng']
    df = df.reindex(columns = columnstitles)
    
    return df

In [ ]:
pacher = readstagedata('Pacher', 16)
kaemna = readstagedata('Kaemna', 16)
reichenbach = readstagedata('Reichenbach', 16)

kweight=65
pweight=63
rweight=64

kaemna.head(10)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (10, 8))
reichenbach.plot(x = 'Distance', y = 'Altitude', ax = ax1, legend = False)
reichenbach.plot(x = 'Distance', y = 'Speed', ax = ax2, legend = False)

ax1.set_xlabel('Distance [km]')
ax1.set_ylabel('Altitude [m]')
ax2.set_xlabel('Distance [km]')
ax2.set_ylabel('Speed [kph]')

fig.subplots_adjust(hspace = 0.2)

In [ ]:
import plotly as py
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from scipy import signal #for smoothing purposes

In [ ]:
def plot_data(kaemna):
    fig = make_subplots(specs = [[{"secondary_y" : True}]])

    #speed trace
    fig.add_trace(
        go.Scatter(
            x = kaemna['Distance'], 
            y = kaemna['Speed'],
            line = dict(color = 'darkturquoise', width = 1),
            name = 'Speed'), 
        secondary_y = False
    )

    #altitude trace
    fig.add_trace(
        go.Scatter(
            x = kaemna['Distance'], 
            y = kaemna['Altitude'],
            line = dict(color = 'black', dash = 'dot'),
            name = 'Altitude'), 
        secondary_y = True
    )

    #smoothed speed trace (Savitzky-Golay filter)
    fig.add_trace(
        go.Scatter(
            x = kaemna['Distance'], 
            y = signal.savgol_filter(kaemna['Speed'], 101, 0), #use a Savitzky-Golay filter to smooth the speed data
            line = dict(color = 'red', width = 3),
            name = 'Smoothed Speed'), 
        secondary_y = False
    )

    fig.update_xaxes(title_text = 'Distance [km]') #x-axis label

    fig.update_yaxes(title_text = 'Speed [kph]', secondary_y = False) #primary y-axis label
    fig.update_yaxes(title_text = 'Altitude [m]', secondary_y = True) #secondary y-axis label

    fig.show()
    
plot_data(kaemna)

In [ ]:
#want to observe the shifts in time, so we plot time vs distance for each rider
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = kaemna["Time"],
        y = kaemna["Distance"],
        line = dict(color = 'black'),
        name = 'Kaemna'
    )
)

fig.add_trace(
    go.Scatter(
        x = reichenbach["Time"],
        y = reichenbach["Distance"],
        line = dict(color = 'red'),
        name = 'Reichenbach'
    )
)

fig.add_trace(
    go.Scatter(
        x = pacher["Time"],
        y = pacher["Distance"],
        line = dict(color = 'green'),
        name = 'Pacher'
    )
)

fig.update_layout(
    xaxis_title = "Time[s]",
    yaxis_title = "Distance[km]"
)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = kaemna["Time"],
        y = kaemna["Altitude"],
        line = dict(color = 'black'),
        name = 'Kaemna'
    )
)

fig.add_trace(
    go.Scatter(
        x = reichenbach["Time"],
        y = reichenbach["Altitude"],
        line = dict(color = 'red'),
        name = 'Reichenbach'
    )
)

fig.add_trace(
    go.Scatter(
        x = pacher["Time"],
        y = pacher["Altitude"],
        line = dict(color = 'green'),
        name = 'Pacher'
    )
)

fig.update_layout(
    xaxis_title = "Time[s]", 
    yaxis_title = "Altitude[m]"
)

From the plots we identify a time gap betwen recordings. Using Kaemna as absolute time, we find time gaps,

Kaemna: +0,
Geschke: +161,
Pacher: +205,
Reichenbach: +549.

In [ ]:
kaemna["Shifted_time"] = kaemna["Time"] - 0
pacher["Shifted_time"] = pacher["Time"] - 206
reichenbach["Shifted_time"] = reichenbach["Time"] - 544

In [ ]:
#add column to each dataframe to identify the rider
kaemna.insert(9, "Rider", "Kaemna")
pacher.insert(9, "Rider", "Pacher")
reichenbach.insert(9, "Rider", "Reichenbach")

In [ ]:
#combine all dataframes into one data frame
frames = [pacher, kaemna, reichenbach]
kaem_pach_reic = pd.concat(frames, sort = False)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Shifted_time"], 
        y = kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Distance"],
        line = dict(color = 'black'),
        name = 'Pacher') 
)

fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Shifted_time"], 
        y = kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Distance"],
        line = dict(color = 'red'),
        name = 'Kaemna') 
)

fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Reichenbach"]["Shifted_time"], 
        y = kaem_pach_reic[kaem_pach_reic["Rider"] == "Reichenbach"]["Distance"],
        line = dict(color = 'green'),
        name = 'Reichenbach') 
)

fig.update_layout(
    xaxis_title = "Shifted time [s]",
    yaxis_title = "Distance [km]"
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Distance"], 
        y = kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Altitude"],
        line = dict(color = 'black'),
        name = 'Pacher') 
)

fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Distance"], 
        y = kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Altitude"],
        line = dict(color = 'red'),
        name = 'Kaemna') 
)

fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Reichenbach"]["Distance"], 
        y = kaem_pach_reic[kaem_pach_reic["Rider"] == "Reichenbach"]["Altitude"],
        line = dict(color = 'green'),
        name = 'Reichenbach') 
)

fig.update_layout(
    xaxis_title = "Shifted time [s]",
    yaxis_title = "Distance [m]"
)

fig.show()

In [ ]:
fig = make_subplots(specs = [[{"secondary_y" : True}]])

fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Distance"], 
        y = signal.savgol_filter(kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Estimated watts"], 1001, 0)/pweight,
        line = dict(color = 'black'),
        name = 'Pacher'),
    secondary_y = False 
)

fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Distance"], 
        y = signal.savgol_filter(kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Estimated watts"], 1001, 0)/kweight,
        line = dict(color = 'red'),
        name = 'Kaemna'),
    secondary_y = False 
)

#fig.add_trace(
 #   go.Scatter(
  #      x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Reichenbach"]["Distance"], 
   #     y = signal.savgol_filter(kaem_pach_reic[kaem_pach_reic["Rider"] == "Reichenbach"]["Estimated watts"], 1001, 0)/rweight,
    #    line = dict(color = 'green'),
     #   name = 'Reichenbach'),
 #   secondary_y = False
#)

#altitude trace
fig.add_trace(
    go.Scatter(
        x = kaemna['Distance'], 
        y = kaemna['Altitude'],
        line = dict(color = 'black', dash = 'dot'),
        name = 'Altitude'), 
    secondary_y = True
)


In [ ]:
fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Distance"], 
        y = signal.savgol_filter(kaem_pach_reic[kaem_pach_reic["Rider"] == "Pacher"]["Speed"], 51, 0),
        line = dict(color = 'black', dash = 'dot'),
        name = 'Pacher'),
    secondary_y = True 
)  

fig.add_trace(
    go.Scatter(
        x = kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Distance"], 
        y = signal.savgol_filter(kaem_pach_reic[kaem_pach_reic["Rider"] == "Kaemna"]["Speed"], 51, 0),
        line = dict(color = 'red', dash = 'dot'),
        name = 'Kaemna'),
    secondary_y = True 
)

fig.update_layout(
    xaxis_title = "Distance [km]",
    yaxis_title = "Power [W]"
)

fig.show()
#y = signal.savgol_filter(kaemna['Speed'], 101, 0), #use a Savitzky-Golay filter to smooth the speed data




In [ ]:
pacher.head(10)